In [1]:
%pip install pandas
%pip install scikit-learn
%pip install imblearn
%pip install numpy
%pip install matplotlib


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

# 1. Đọc dữ liệu từ file CSV
data = pd.read_csv('./dataset/Amazon_Product_Review_cleaned.csv')

# Hiển thị các cột của dataset (để kiểm tra)
print("Các cột trong dataset:", data.columns.tolist())

# 2. Tiền xử lý văn bản trong cột 'review'
def preprocess_text(text):
    text = str(text)            # Đảm bảo text là chuỗi
    text = text.lower()         # Chuyển về chữ thường
    # Giữ lại các ký tự chữ cái (bao gồm có dấu), số và dấu cách
    text = re.sub(r'[^a-zA-Z0-9À-ỹ\s]', '', text)
    return text

# Áp dụng tiền xử lý cho cột review
data['review'] = data['review_headline'].apply(preprocess_text)

# 3. Loại bỏ các dòng có giá trị bị thiếu
data = data.dropna(subset=['review', 'star_rating'])

# 4. Đảm bảo cột star_rating ở dạng số nguyên
data['star_rating'] = data['star_rating'].astype(int)

# Hiển thị các giá trị star_rating độc nhất
print("Các star_rating độc nhất:", data['star_rating'].unique())

# 5. Tách dữ liệu thành đặc trưng (X) và nhãn (y)
X = data['review']
y = data['star_rating']

# 6. Chia dữ liệu thành tập huấn luyện và tập kiểm tra (80% huấn luyện, 20% kiểm tra)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 7. Trích xuất đặc trưng từ văn bản bằng TF-IDF
tfidf = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

# # 9. Áp dụng SMOTE để cân bằng dữ liệu lớp thiểu số
# X_train_dense = X_train_tfidf.toarray()  # Chuyển đổi ma trận thưa thành ma trận đặc
# smote = SMOTE(random_state=42)
# X_train_resampled, y_train_resampled = smote.fit_resample(X_train_dense, y_train)

# 8. Xây dựng và huấn luyện mô hình SVM (phân loại đa lớp: 1,2,3,4,5)
svm_model = SVC(kernel='rbf', random_state=42)
svm_model.fit(X_train_tfidf, y_train)
# # 10. Dự đoán nhãn cho tập kiểm tra
# print("Số lượng mẫu trước SMOTE:", Counter(y_train))
# print("Số lượng mẫu sau SMOTE:", Counter(y_train_resampled))

# # Define the pipeline
# pipeline = Pipeline([
#     ('tfidf', TfidfVectorizer(max_features=5000)),
#     ('svm', SVC(kernel='rbf', random_state=42))
# ])

# cv_scores = cross_val_score(pipeline, X, y, cv=2, scoring='accuracy')
# print("Cross-validation accuracy scores:", cv_scores)
# print("Mean CV accuracy:", cv_scores.mean())
# print("Mean CV accuracy:", cv_scores.mean())




# Khi chạy với côtej 'review_body' thì kết quả là:
# không có GridSearchCV + Smote + Hyperparameter, kernel: linear -> Accurancy: 0.71
# không có GridSearchCV + Smote + Hyperparameter, kernel: rbf -> Accurancy: 0.70
# Có GridSearchCV + Smote + Hyperparameter, kernel: linear class_weight='balanced' -> Accurancy: 0.6-0.64
# Có GridSearchCV + Smote + Hyperparameter, kernel: rbf class_weight='balanced' -> Accurancy: 0.64

# Khi chạy với cột 'review_headline' thì kết quả là:
# không có GridSearchCV + Smote + Hyperparameter, kernel: linear -> Accurancy: 0.77
# không có GridSearchCV + Smote + Hyperparameter, kernel: rbf -> Accurancy: 0.77
# Có GridSearchCV + Smote + Hyperparameter, kernel: rbf class_weight='balanced' -> Accurancy: 0.77




Các cột trong dataset: ['marketplace', 'customer_id', 'review_id', 'product_id', 'product_parent', 'product_title', 'product_category', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase', 'review_headline', 'review_body', 'review_date', 'sentiment']
Các star_rating độc nhất: [5 1 3 4 2]


SVC(random_state=42)

In [3]:
# 9. Dự đoán và đánh giá mô hình
y_pred = svm_model.predict(X_test_tfidf)

print("Độ chính xác (Accuracy):", accuracy_score(y_test, y_pred), "\n")
print("Ma trận nhầm lẫn:")
print(confusion_matrix(y_test, y_pred), "\n")
print("Báo cáo phân loại:")
print(classification_report(y_test, y_pred))

Độ chính xác (Accuracy): 0.7724473257698541 

Ma trận nhầm lẫn:
[[ 164    3   32   14  131]
 [  34   50   30   20  103]
 [  31    5  167   54  205]
 [  12    0   34  506  607]
 [  15    2   15   57 3879]] 

Báo cáo phân loại:
              precision    recall  f1-score   support

           1       0.64      0.48      0.55       344
           2       0.83      0.21      0.34       237
           3       0.60      0.36      0.45       462
           4       0.78      0.44      0.56      1159
           5       0.79      0.98      0.87      3968

    accuracy                           0.77      6170
   macro avg       0.73      0.49      0.55      6170
weighted avg       0.77      0.77      0.74      6170



In [4]:

# 9. Kiểm thử với các câu thực tế
test_sentences = [
    # 30% tiếng Việt (3/10)
    "Sản phẩm rất tốt, tôi hoàn toàn hài lòng!",
    "Không đáng tiền, rất thất vọng về chất lượng.",
    "Giao hàng nhanh, đóng gói cẩn thận và dịch vụ hậu mãi tốt.",
    
    # 30% tiếng Anh (3/10)
    "Amazing quality, definitely worth it!",
    "Terrible experience, will never buy again.",
    "I am extremely dissatisfied with this product.",
    
    # 40% mix (4/10)
    "Sản phẩm rất tốt nhưng delivery lại quá chậm, cần cải thiện.",
    "I love the design, nhưng chất lượng lại không ổn như quảng cáo.",
    "Great product và dịch vụ khách hàng cũng rất tốt, tôi khuyên mọi người nên thử.",
    "This is a decent product nhưng tôi mong đợi nhiều hơn về tính năng.",
    
    # Thêm 5 câu dài hơn
    "Sau nhiều lần thử nghiệm và sử dụng sản phẩm, tôi nhận thấy rằng dù có thiết kế hiện đại và bắt mắt, nhưng sản phẩm lại liên tục gặp phải các vấn đề kỹ thuật khiến trải nghiệm của tôi bị gián đoạn nghiêm trọng, và dịch vụ hỗ trợ khách hàng cũng không thể giải quyết các vấn đề đó một cách kịp thời.",
    "After several months of use, the product started showing persistent performance issues and unexpected malfunctions, which led me to contact customer service, but unfortunately, their responses were slow and unhelpful, leaving me deeply disappointed.",
    "Tôi thực sự ấn tượng với thiết kế của sản phẩm, however, when it came to performance, nó đã không đáp ứng được kỳ vọng của tôi vì có quá nhiều lỗi xảy ra, especially khi sử dụng trong môi trường làm việc căng thẳng.",
    "The initial impression of the product was excellent with a sleek design and promising features, nhưng sau khi sử dụng một thời gian, các vấn đề về ổn định và hiệu suất đã dần trở nên rõ ràng, making me regret my purchase despite the appealing presentation.",
    "Sản phẩm được quảng cáo với nhiều ưu điểm vượt trội, tuy nhiên, sau khi mua và sử dụng trong thời gian dài, tôi nhận ra rằng nó lại có nhiều hạn chế nghiêm trọng về độ bền, hiệu suất, cũng như dịch vụ hậu mãi kém, dẫn đến trải nghiệm không như mong đợi và làm giảm đáng kể sự tin tưởng của tôi vào thương hiệu."
]

X_test_sentences = tfidf.transform(test_sentences)
y_test_pred = svm_model.predict(X_test_sentences)

# Hiển thị kết quả
for text, pred in zip(test_sentences, y_test_pred):
    print(f"Review: {text} -> Dự đoán: {pred} sao")

Review: Sản phẩm rất tốt, tôi hoàn toàn hài lòng! -> Dự đoán: 5 sao
Review: Không đáng tiền, rất thất vọng về chất lượng. -> Dự đoán: 3 sao
Review: Giao hàng nhanh, đóng gói cẩn thận và dịch vụ hậu mãi tốt. -> Dự đoán: 5 sao
Review: Amazing quality, definitely worth it! -> Dự đoán: 5 sao
Review: Terrible experience, will never buy again. -> Dự đoán: 1 sao
Review: I am extremely dissatisfied with this product. -> Dự đoán: 5 sao
Review: Sản phẩm rất tốt nhưng delivery lại quá chậm, cần cải thiện. -> Dự đoán: 4 sao
Review: I love the design, nhưng chất lượng lại không ổn như quảng cáo. -> Dự đoán: 3 sao
Review: Great product và dịch vụ khách hàng cũng rất tốt, tôi khuyên mọi người nên thử. -> Dự đoán: 5 sao
Review: This is a decent product nhưng tôi mong đợi nhiều hơn về tính năng. -> Dự đoán: 4 sao
Review: Sau nhiều lần thử nghiệm và sử dụng sản phẩm, tôi nhận thấy rằng dù có thiết kế hiện đại và bắt mắt, nhưng sản phẩm lại liên tục gặp phải các vấn đề kỹ thuật khiến trải nghiệm của tôi 